In [1]:
\cd /home/qenv/

In [1]:
\l model/init.q

[0;31mmodel/init.q. OS reports: No such file or directory[0m: [0;31mmodel/init.q. OS reports: No such file or directory[0m

In [2]:
\ls

"Dockerfile"
"README.md"
"TODO.md"
"__init__.py"
"atari.sh"
"binance.q"
"common"
"config"
"engine"
"env.q"
"envTest.q"
"ingest"
"kdb.sh"
"kdbj.sh"
"notebooks"
"pipeline.q"
"qenv_cluster.sh"
"quantest"
"research"
"server.q"
"serverTest.q"
"state"
..


In [5]:
a:(1b;1;();(
  `oId`acc`ivn`price`okind`state`oqty`dqty`einst!(0;0;0;1000;0;0;100;100;0);
  `oId`acc`ivn`price`okind`state`oqty`dqty`einst!(1;0;0;1000;0;0;100;100;0);
  `oId`acc`ivn`price`okind`state`oqty`dqty`einst!(2;0;0;1000;0;0;100;100;0)
    ))

In [15]:
a:(1b;1;();(
                  `price`side`qty`hqty`iqty`vqty!(1000;1;1000;1000;1000;1000);
                  `price`side`qty`hqty`iqty`vqty!(1000;1;1000;1000;1000;1000);
                  `price`side`qty`hqty`iqty`vqty!(1000;1;1000;1000;1000;1000)
                )); // GetLevel 
b:(1b;1;();(
                  `oId`side`acc`ivn`price`okind`state`oqty`dqty`einst!(0;-1;0;0;1000;0;0;100;100;0);
                  `oId`side`acc`ivn`price`okind`state`oqty`dqty`einst!(1;-1;0;0;1000;0;0;100;100;0);
                  `oId`side`acc`ivn`price`okind`state`oqty`dqty`einst!(2;-1;0;0;1000;0;0;100;100;0)
                )); // GetOrder 


In [16]:
b[3]

oId side acc ivn price okind state oqty dqty einst
--------------------------------------------------
0   -1   0   0   1000  0     0     100  100  0    
1   -1   0   0   1000  0     0     100  100  0    
2   -1   0   0   1000  0     0     100  100  0    


In [17]:
update oprice:price, oside:side from b[3]

oId side acc ivn price okind state oqty dqty einst oprice oside
---------------------------------------------------------------
0   -1   0   0   1000  0     0     100  100  0     1000   -1   
1   -1   0   0   1000  0     0     100  100  0     1000   -1   
2   -1   0   0   1000  0     0     100  100  0     1000   -1   


In [18]:
ij[`price xkey a[3];`price xgroup (update oprice:price, oside:side from b[3])]

price| side     qty  hqty iqty vqty oId   acc   ivn   okind state oqty       ..
-----| ----------------------------------------------------------------------..
1000 | -1 -1 -1 1000 1000 1000 1000 0 1 2 0 0 0 0 0 0 0 0 0 0 0 0 100 100 100..
1000 | -1 -1 -1 1000 1000 1000 1000 0 1 2 0 0 0 0 0 0 0 0 0 0 0 0 100 100 100..
1000 | -1 -1 -1 1000 1000 1000 1000 0 1 2 0 0 0 0 0 0 0 0 0 0 0 0 100 100 100..


In [ ]:
s:0!{$[x>0;desc[y];asc[y]]}[neg[side];]; 